# OpenAI Responses API

## What is the OpenAI Responses API?

The Responses API is a new API released in March 2025. It is a combination of the traditional 
Chat Completions API and the Assistants API, providing support for:

- **Traditional Chat Completions:** Facilitates seamless conversational AI experiences.
- **Web Search:** Enables real-time information retrieval from the internet.
- **File Search:** Allows searching within files for relevant data.

Accordingly, the Assistants API will be retired in 2026. 

> **For new users, OpenAI recommends using the Responses API instead of the Chat Completions API to leverage its expanded capabilities.**

For a comprehensive comparison between the Responses API and the Chat Completions API, refer to the official OpenAI documentation: 
[Responses vs. Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

## Summary of This Notebook
This notebook provides a hands-on guide for using the **OpenAI Responses API** to analyze tweets. 
It covers essential techniques such as:

- **Creating a vector store** and uploading tweets for semantic search.
- **Using file search** to analyze private datasets.
- **Performing a web search** to retrieve the latest public information.
- **Utilizing stateful responses** to maintain conversation context.
- **Combining file and web search** to enhance retrieval-augmented generation (RAG) applications.

By the end of this notebook, users will be able to integrate OpenAI's Responses API for efficient data retrieval and analysis of structured and unstructured data.

## Install Required Libraries
To use the OpenAI Responses API, we need to install the following libraries:

- **`openai`**: Provides access to OpenAI's APIs, including the Responses API

In [1]:
pip install openai -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.3.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

In [2]:
from IPython.display import Markdown, display
import boto3
from botocore.exceptions import ClientError
import json
import io

## Retrieve Secrets from AWS Secrets Manager

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Initialize OpenAI Client

In [4]:
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']

client = OpenAI(api_key=openai_api_key)

## File Search API

### Introduction to File Search
File search API enables efficient retrieval of relevant information 
from uploaded files by leveraging vector-based indexing. This feature is particularly useful 
for searching large datasets, extracting insights, and improving retrieval-augmented generation (RAG) applications.

Unlike traditional keyword-based searches, the Responses API uses embeddings 
to identify semantically relevant content, making it ideal for analyzing structured 
and unstructured text data (OpenAI, 2025).

For more details, visit the official OpenAI documentation: 
[File Search in Responses API](https://platform.openai.com/docs/guides/tools-file-search).

### Create a Vector Store

In [5]:
vector_store = client.vector_stores.create(
    name="my_vector_store"
)
vector_store_id = vector_store.id
print(vector_store_id)

vs_691264a2a1308191be1b1dfc13ecad44


### Upload Files

In [6]:
with open('tweet_text.json', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-WA7damSVacnRiWPjVRKMvW


### Attach File to Vector Store

In [7]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-WA7damSVacnRiWPjVRKMvW


### Query the Vector Store

In [21]:
query = "the latest development in generativeAI"

In [22]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1f1
 Relevant score: 0.6010846063542131
Learning Plans. Use coupon code 𝐒𝐏𝐋𝟑𝟎 at checkout.🎯\n\nJoin Now 👉 https://t.co/LS2JuCrVmz\n\n#AI #Ce
 Relevant score: 0.5987078953443187
Create stunning, cinematic videos from a prompt—now with audio, physics, and cameos. One prompt = en
 Relevant score: 0.5911156596667607
They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1d1
 Relevant score: 0.5622353955032359
🤤 AI is making cinematic food commercials in minutes. The sizzling patty, the dripping cheese... no 
 Relevant score: 0.551242779477055


## OpenAI Response API

### Simple Response

In [23]:
simple_response = client.responses.create(
  model="gpt-4o",
  input=[
      {
          "role": "user",
          "content": query
      }
  ]
)

In [24]:
display(Markdown(simple_response.output_text))

As of 2023, there have been several exciting developments in generative AI:

1. **Multimodal Models**: Advanced AI systems that can process and generate text, images, audio, and video, such as OpenAI's GPT-4 and its successors, offer enhanced capabilities by integrating different forms of media.

2. **Improved Text Generation**: Models capable of creating more coherent and contextually relevant content with better understanding of nuances in language, leading to applications in content creation, customer support, and creative writing.

3. **Image and Art Generation**: Tools like DALL-E and Midjourney enable the creation of highly detailed and imaginative artwork, supporting artists and designers in various creative fields.

4. **Ethical AI Advances**: Greater emphasis on responsible AI development with frameworks to address biases, ensure fairness, and improve transparency in generative models.

5. **Real-time AI Applications**: Breakthroughs in reducing latency allow for real-time applications of generative AI in gaming, virtual reality, and personalized content generation.

6. **Domain-specific Models**: Tailored generative models for specific industries, such as pharmaceuticals for drug discovery or finance for market analysis, optimizing outputs for specialized needs.

7. **Collaboration with Humans**: Tools designed to assist rather than replace human creators, enhancing creativity through AI-driven suggestions and collaborative workflows.

These developments underscore the rapid evolution of generative AI, highlighting its expanding role across numerous sectors.

### File Search Response

In [25]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [26]:
display(Markdown(file_search_response.output_text))


The latest developments in generative AI include several exciting advancements:

1. **OpenAI's Sora2**: This tool allows users to create cinematic videos from a prompt, now with added features like audio, physics, and cameos, offering endless creativity for creators.

2. **GPT-5 and DALL·E 4**: These models are being used to create unique multimedia experiences, such as combining quantum fractals with music generation.

3. **Generative AI in Business**: Companies like Atos are using generative AI for supply chain disruption analysis, integrating it with platforms like SAP and AWS to assess risks and boost resilience.

4. **AI in Creative Industries**: Generative AI is transforming creative fields by enabling the production of content like cinematic food commercials without traditional sets or crews.

5. **Enterprise Applications**: IBM's Watsonx is bringing generative AI to enterprises, allowing teams to build custom large language models to enhance customer engagement and streamline processes.

These developments highlight the diverse applications and transformative potential of generative AI across various sectors.

## Web Search API

### Introduction to Web Search
The OpenAI Web Search tool allows models to retrieve real-time information from the internet. 
This capability is particularly useful for obtaining up-to-date data, fact-checking, and expanding knowledge 
without relying solely on pre-trained information. 

By leveraging OpenAI's web search functionality, the Responses API can fetch external data 
and provide accurate, relevant results in real time (OpenAI, 2025). 
This feature enhances applications that require the latest insights, such as news aggregation, research, 
or dynamic content generation.

For more details, visit the official OpenAI documentation: 
[Web Search in Responses API](https://platform.openai.com/docs/guides/tools-web-search).

### Perform Web Search

In [27]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [28]:
display(Markdown(web_search_response.output_text))

Here’s an in‑depth, structured update on the **latest developments in generative AI** as of **November 10, 2025**, encompassing both industry milestones and academic innovations.

---

##  1. Cutting‑Edge Model Releases & Deployments

- **OpenAI** released **GPT‑5** on **August 7, 2025**—a multimodal foundational model that powers ChatGPT and Microsoft’s Copilot. It delivers improved performance, safety enhancements, and is widely accessible via API and free-tier usage limits.([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5?utm_source=openai))

- Also noteworthy, **OpenAI unveiled “Aardvark”**—a **GPT‑5–powered autonomous security researcher**. Currently in private beta, Aardvark identifies software vulnerabilities with ~92% recall and has already contributed to 10 CVE‑tracked disclosures.([aiwebbiz.com](https://aiwebbiz.com/blog/top-5-ai-news-of-the-week-november-2025/?utm_source=openai))

- **Google DeepMind’s Gemini family** continues advancing. Notable within this is **“Nano Banana”** (Gemini 2.5 Flash Image), introduced in **August 2025**. This text-to-image editor went viral for generating photorealistic 3D-figurine-style images and supports features like multi-image fusion and SynthID watermarking.([en.wikipedia.org](https://en.wikipedia.org/wiki/Nano_Banana?utm_source=openai))

- A **Nano Banana 2** is now on the horizon, expected to launch within weeks. It promises faster generation, enhanced stylistic diversity, and deeper integration with the upcoming Gemini 3.0 series.([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

---

##  2. Major Product Integrations & Ecosystem Expansion

- **OpenAI’s Sora**, a video-gen model, saw explosive adoption: ~470,000 Android downloads on its first day and ~360,000 on iOS. Its second iteration, **Sora 2**, was launched on **October 1, 2025**, bringing synchronized audio, cinematic styles, and enhanced consistency across clips.([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

- **Google Gemini AI** is deepening its utility:
  - Introduced **Deep Research**—powerful multimodal querying across Gmail, Drive, Chat, and standard search to generate contextual, personalized insights.([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))
  - A **Gemini 3 Pro preview** appeared in **Vertex AI**, expected to arrive soon with a milestone 1-million-token context window for more complex tasks.([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

---

##  3. Industry Players Expanding AI Capabilities

- **Qualcomm** unveiled two new AI inference accelerators—**AI200** and **AI250**—designed for data center deployments by 2026–2027. Features include high memory bandwidth, scalable design, and compatibility with PyTorch/ONNX, positioning Qualcomm as a competitive force against NVIDIA and AMD.([tomshardware.com](https://www.tomshardware.com/tech-industry/artificial-intelligence/qualcomm-unveils-ai200-and-ai250-ai-inference-accelerators-hexagon-takes-on-amd-and-nvidia-in-the-booming-data-center-realm?utm_source=openai))

- **Google** launched the **TIME AI Agent**, developed with Scale AI. Released today (November 10, 2025), it’s a generative-AI-powered platform enabling interactive news consumption — from summarization to audio reports and translations, while maintaining editorial integrity.([time.com](https://time.com/7332572/the-story-behind-the-time-ai-agent/?utm_source=openai))

- **Google** also announced a **Generative AI Leader certification for veterans**, starting **November 13–14, 2025**, part of a broader push to retrain service members for tech careers using generative AI skills.([startuphub.ai](https://www.startuphub.ai/ai-news/ai-research/2025/google-accelerates-veteran-ai-training-initiatives/?utm_source=openai))

- **BrainChip**, an Australian neuromorphic AI company, raised A$35 million (with an additional A$2 million planned) to accelerate its **Akida edge AI platform** and next-gen products.([sharecafe.com.au](https://www.sharecafe.com.au/2025/11/10/brainchip-announces-a35-million-placement-and-share-purchase-plan/?utm_source=openai))

---

##  4. Quantum-Enhanced Generative AI Research

- A landmark study published in **September 2025** demonstrates **generative quantum advantage** using a 68‑qubit superconducting processor. These quantum models can learn and generate classically intractable distributions, marking a breakthrough toward quantum-enhanced generative AI.([arxiv.org](https://arxiv.org/abs/2509.09033?utm_source=openai))

---

##  5. Key Themes & Strategic Overview

- **Multimodality continues to dominate**: Models like GPT‑5, Gemini image variants, and Sora 2 integrate text, image, video, and audio in unified interfaces—pushing toward more immersive and versatile AI.([gartner.com](https://www.gartner.com/en/newsroom/press-releases/2024-09-09-gartner-predicts-40-percent-of-generative-ai-solutions-will-be-multimodal-by-2027?utm_source=openai))

- **Agentic intelligence emerges**: Features like Aardvark (autonomous security detection), Gemini Deep Research, and Sora 2’s planning-oriented design reflect a shift from passive AI to systems that proactively operate within workflows.([aiwebbiz.com](https://aiwebbiz.com/blog/top-5-ai-news-of-the-week-november-2025/?utm_source=openai))

- **Infrastructure and regulation focus**: Industry players are bolstering infrastructure (Qualcomm’s accelerators) and talent pipelines (Google’s veteran program). Meanwhile, deployment at scale continues with attention on governance and editorial integrity (TIME AI Agent).([startuphub.ai](https://www.startuphub.ai/ai-news/ai-research/2025/google-accelerates-veteran-ai-training-initiatives/?utm_source=openai))

---

### Summary Snapshot

| Category                     | Highlight                                                                   |
|------------------------------|-----------------------------------------------------------------------------|
| Model Releases               | GPT‑5, Aardvark, Nano Banana, Nano Banana 2 (upcoming), Sora 2              |
| Ecosystem Integrations       | Gemini Deep Research, Vertex AI glimpse of Gemini 3 Pro                    |
| Infrastructure & Access      | Qualcomm AI200/250, TIME AI Agent, veteran certification, BrainChip funding |
| Research Breakthroughs       | Quantum generative models with demonstrated advantage                      |

---

If you’d like to dive deeper into any specific development—be it performance benchmarks, deployment timelines, or comparative analyses—just let me know!

### Stateful Response

The OpenAI Responses API includes a stateful feature that enables continuity in interactions. 
By using the `response_id`, a conversation can persist across multiple queries, 
allowing users to refine or expand upon previous searches. This is particularly useful for iterative research, 
dynamic content generation, and applications that require follow-up queries based on prior responses.

In [29]:
fetched_response = client.responses.retrieve(response_id=web_search_response.id)
display(Markdown(fetched_response.output_text[:100]))

Here’s an in‑depth, structured update on the **latest developments in generative AI** as of **Novemb

### Continue Query with Web Search

In [30]:
continue_query = 'find different news'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [31]:
display(Markdown(continue_search_response.output_text))

Here’s a **detailed, up-to-date**, and well-structured overview of the latest developments in **generative AI** as of **November 10, 2025**. These updates reflect news through early November and cover industry-leading model releases, product integrations, hardware, policy, and academic advances. Each statement is clearly cited for accuracy.

---

##  Generative AI: Key Breakthroughs and Product Launches

### OpenAI and Agentic AI
- **OpenAI unveiled “Aardvark,”** a GPT-5-powered agentic security researcher in private beta on **October 30, 2025**. It autonomously discovers software vulnerabilities with about 92% recall and has contributed to multiple CVE disclosures ([aiwebbiz.com](https://aiwebbiz.com/blog/top-5-ai-news-of-the-week-november-2025/?utm_source=openai)).
- **Sora video app** experienced explosive uptake, with approximately 470,000 downloads on Android and 360,000 on iOS on its first day (November 6) ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).

### Google’s Expanding Ecosystem
- **Gemini’s “Deep Research”** now empowers users to query Gmail, Drive, Chat, and standard search, generating high-context, customized results ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).
- A **preview of Gemini 3 Pro** appeared on Vertex AI, expected to deliver a massive 1 million-token context window for advanced developer use ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).
- **Veo 3.1**, the latest update to Google’s text-to-video model, enhances narrative control, enables extended clip generation via keyframe interpolation, and is available through paid preview via the Gemini API ([voxfor.com](https://www.voxfor.com/what-is-new-in-ai-the-latest-news-from-october-2025/?utm_source=openai)).
- **Gemini Nano Banana 2** is slated for release soon, promising faster image generation and a broader stylistic range, integrated with the upcoming Gemini 3.0 series ([medium.com](https://medium.com/%40CherryZhouTech/ai-news-november-1-7-2025-10-ai-advances-you-cant-miss-this-week-e2e86d331bd0?utm_source=openai)).
- In **October 2025**, Google rolled out numerous AI innovations including a verifiable quantum algorithm with 13,000× speed-up, **Gemini for Home**, **Gemini Enterprise** for secure workplace AI, **vibe coding** in AI Studio, Flow editing enhancements, AI-powered updates to Google Home and Android XR devices, and the launch of a learning platform, Google Skills ([blog.google](https://blog.google/technology/ai/google-ai-updates-october-2025/?utm_source=openai)).

### Anthropic and Agentic Efficiency
- **Claude 3.7 Sonnet**, launched earlier in 2025, offers a hybrid reasoning mode called “extended thinking,” designed for tasks like coding, physics, and math. It notably costs less than OpenAI’s comparable models ([reuters.com](https://www.reuters.com/technology/artificial-intelligence/anthropic-launches-advanced-ai-hybrid-reasoning-model-2025-02-24/?utm_source=openai)).
- Additionally, **Anthropic introduced Claude Haiku 4.5**, a lightweight but high-performing model delivering GPT-5-level reasoning and coding power at 4–5× speed and significantly lower cost. It supports a 200K token context window and agentic function execution, ideal for automation tasks ([voxfor.com](https://www.voxfor.com/what-is-new-in-ai-the-latest-news-from-october-2025/?utm_source=openai)).

### Creative Tools and Multimedia AI
- **Adobe MAX 2025** revealed the **Firefly Image Model 5**, enabling higher-quality image and video creation across Adobe Express, Firefly, Creative Cloud, and GenStudio. Integration with Google’s Nano Banana was also announced ([timesofindia.indiatimes.com](https://timesofindia.indiatimes.com/technology/tech-news/adobe-launches-ai-audio-video-and-photo-tools-across-express-firefly-and-creative-cloud-brings-firefly-image-model-5-googles-nano-banana-and-more/articleshow/124876149.cms?utm_source=openai)).
- Earlier in the year, Adobe released **Firefly Image Model 4 and 4 Ultra**, offering enhanced speed, realism, customization, high-resolution capabilities, plus a new collaborative moodboarding tool, Firefly Boards, in public beta ([theverge.com](https://www.theverge.com/news/655230/adobe-ai-firefly-image-model-4-availability?utm_source=openai)).
- **Runway Gen-4**, released March 31, 2025, is a text-to-video model that generates up to 10-second clips using transformer and diffusion architecture. While promising, it still shows limitations in character consistency and motion realism ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gen-4_%28AI_image_and_video_model%29?utm_source=openai)).

---

##  Hardware Advancements & Compute Power Race

- **Qualcomm** introduced **AI200 and AI250** inference accelerators designed for data centers. AI200 is expected in 2026 with AI250 following in 2027. They offer features like micro-tile inferencing, high memory bandwidth, Gen AI model encryption, and compatibility with AI frameworks ([tomshardware.com](https://www.tomshardware.com/tech-industry/artificial-intelligence/qualcomm-unveils-ai200-and-ai250-ai-inference-accelerators-hexagon-takes-on-amd-and-nvidia-in-the-booming-data-center-realm?utm_source=openai)).
- **NVIDIA**, at GTC 2025, launched next-gen chips: **Blackwell Ultra** (late 2025), **Vera Rubin** (late 2026), and **Rubin Ultra** (2027). The company also presented the Cosmos video model for training data generation, the Isaac GR00T N1 robotics model, the Halos autonomous safety system, and the Newton physics engine co-developed with DeepMind and Disney Research ([apnews.com](https://apnews.com/article/457e9260aa2a34c1bbcc07c98b7a0555?utm_source=openai)).

---

##  Regulatory, Industry, and Business Integration

- **Regulatory and educational moves**: NTT Data (Japan) announced plans to train its 200,000 employees in generative AI literacy ([ninjaai.com](https://www.ninjaai.com/top-ai-news-for-october-30-2025?utm_source=openai)).
- **Legal confrontations**: Palantir filed suit against former employees for alleged AI trade secret theft ([ninjaai.com](https://www.ninjaai.com/top-ai-news-for-october-30-2025?utm_source=openai)).
- **Geopolitical collaborations**: Google and NextEra are reviving an Iowa nuclear plant to power AI data centers ([ninjaai.com](https://www.ninjaai.com/top-ai-news-for-october-30-2025?utm_source=openai)).
- **EU regulatory updates**: New transparency and audit rules for generative AI systems; Vietnam is drafting similar legislation ([ninjaai.com](https://www.ninjaai.com/top-ai-news-for-october-30-2025?utm_source=openai)).

---

##  Trends & Strategic Shifts

- **Agentic AI is rising**: Models like Aardvark, DS STAR, Claude Haiku 4.5, and Gemini Enterprise demonstrate AI evolving from reactive tools to proactive agents ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).
- **Multimodal expansion continues**: Veo models, Nano Banana 2, Sora, Gemini’s multimodal integrations, and Firefly’s suite show strong push toward integrating text, image, video, and audio generation ([theverge.com](https://www.theverge.com/news/655230/adobe-ai-firefly-image-model-4-availability?utm_source=openai)).
- **Hardware scaling intensifies**: Investments from Qualcomm and NVIDIA are gearing up to meet growing generative AI compute demands ([tomshardware.com](https://www.tomshardware.com/tech-industry/artificial-intelligence/qualcomm-unveils-ai200-and-ai250-ai-inference-accelerators-hexagon-takes-on-amd-and-nvidia-in-the-booming-data-center-realm?utm_source=openai)).
- **AI regulation and ethics gaining ground**: Legal tensions and new compliance frameworks reflect growing concern over IP and transparency ([ninjaai.com](https://www.ninjaai.com/top-ai-news-for-october-30-2025?utm_source=openai)).

---

### Summary Table

| Category                 | Highlights |
|--------------------------|------------|
| Agentic & Security AI    | Aardvark, Claude Haiku 4.5, DS STAR |
| Video & Image Generation | Veo 3.1, Gemini Nano Banana 2, Sora, Firefly 5 |
| Hardware & Infrastructure| Qualcomm AI200/250, NVIDIA Blackwell/Rubin |
| Regulation & Integration | NTT Data training, legal cases, energy infrastructure, regulations |

---

If you'd like further exploration—such as breakdowns of model performance, enterprise deployment case studies, or policy comparisons—just let me know!

### Combining File Search and Web Search

This is an example of using file search to analyze private data and web search to retrieve public or the latest data. 
The Responses API allows developers to integrate these tools to enhance retrieval-augmented generation (RAG) applications. 
By combining file search with web search, users can leverage structured internal knowledge while also retrieving real-time 
information from external sources, ensuring comprehensive and up-to-date responses. 

In [32]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [20]:
display(Markdown(combined_search_response.output_text))

Here’s a comprehensive and up-to-date overview of the **latest developments in generative AI** as of November 10, 2025. This analysis draws on recent news, academic findings, and industry releases to highlight key breakthroughs across models, multimodal capabilities, hardware, open-source access, and real-world applications.

---

## 1. Cutting-Edge Generative AI Models

- **OpenAI GPT‑5**  
  Released on August 7, 2025, GPT‑5 is a multimodal foundation model that integrates reasoning and non-reasoning capabilities under a unified interface. It is accessible via ChatGPT, Microsoft Copilot, and the OpenAI API, and represents the current state-of-the-art in performance benchmarks. ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5?utm_source=openai))

- **OpenAI o4‑mini**  
  Launched on April 16, 2025, o4‑mini is a lightweight, multimodal reasoning model available to all ChatGPT users. It supports both text and image inputs, including whiteboard sketch analysis, and includes a higher-accuracy “o4‑mini‑high” variant for paid users. ([en.wikipedia.org](https://en.wikipedia.org/wiki/OpenAI_o4-mini?utm_source=openai))

- **Anthropic Claude 3.7 Sonnet**  
  Introduced in February 2025, this hybrid reasoning model offers an “extended thinking mode” that enables self-reflection before responding, improving performance in math, physics, coding, and other complex tasks. It is available across all Claude plans, with the extended mode reserved for paid tiers. ([reuters.com](https://www.reuters.com/technology/artificial-intelligence/anthropic-launches-advanced-ai-hybrid-reasoning-model-2025-02-24/?utm_source=openai))

- **OpenAI’s Open-Weight Models (gpt‑oss‑20b and gpt‑oss‑120b)**  
  In August 2025, OpenAI released its first open-weight models since GPT‑2. These models support chain-of-thought reasoning and can run locally—gpt‑oss‑20b on consumer hardware (e.g., Snapdragon PCs), and gpt‑oss‑120b on more powerful setups like NVIDIA RTX GPUs. ([windowscentral.com](https://www.windowscentral.com/artificial-intelligence/openai-chatgpt/openai-launches-two-gpt-models-theyre-not-gpt-5-but-they-run-locally-on-snapdragon-pcs-and-nvidia-rtx-gpus?utm_source=openai))

---

## 2. Multimodal and Creative AI Innovations

- **Google DeepMind’s “Nano Banana” (Gemini 2.5 Flash Image)**  
  Released on August 26, 2025, this image generation and editing model went viral for its photorealistic “3D figurine” outputs. It supports features like subject consistency, multi-image fusion, and SynthID watermarking, and has driven over 10 million new users to the Gemini app. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Nano_Banana?utm_source=openai))

- **Generative AI for Multimedia**  
  In 2025, models have increasingly unified text, image, audio, and video generation. Notable examples include:
  - **Google’s Veo 3**: A text-to-video model with integrated audio generation. ([aiinsight.blog](https://aiinsight.blog/generative-ai-advancements-in-2025?utm_source=openai))
  - **OpenAI’s Sora**: A video generation tool integrated into ChatGPT for Plus users. ([aiinsight.blog](https://aiinsight.blog/generative-ai-advancements-in-2025?utm_source=openai))
  - **Midjourney V7**: Adds 3D model generation and video capabilities. ([aiinsight.blog](https://aiinsight.blog/generative-ai-advancements-in-2025?utm_source=openai))
  - **Runway Gen‑4**: Introduces “Character Lock” to maintain character consistency across video scenes. ([aiinsight.blog](https://aiinsight.blog/generative-ai-advancements-in-2025?utm_source=openai))
  - **Stability AI’s Stable Audio 2.0**: Enhances AI music generation with better structure and emotional control. ([aiinsight.blog](https://aiinsight.blog/generative-ai-advancements-in-2025?utm_source=openai))

---

## 3. Hardware and Infrastructure Advances

- **NVIDIA Rubin and Blackwell Chips**  
  At GTC 2025, NVIDIA unveiled its next-gen AI chips: Blackwell Ultra (launching late 2025) and Rubin (late 2026), with Rubin Ultra following in 2027. These chips are designed to support generative and agentic AI, robotics, and physical AI applications. ([apnews.com](https://apnews.com/article/457e9260aa2a34c1bbcc07c98b7a0555?utm_source=openai))

- **Qualcomm AI200 and AI250 Accelerators**  
  Announced in late October 2025, these upcoming AI inference accelerators target data center workloads. The AI200 (2026) and AI250 (2027) offer high memory capacity, advanced NPUs, and features like micro-tile inferencing and model encryption. ([tomshardware.com](https://www.tomshardware.com/tech-industry/artificial-intelligence/qualcomm-unveils-ai200-and-ai250-ai-inference-accelerators-hexagon-takes-on-amd-and-nvidia-in-the-booming-data-center-realm?utm_source=openai))

---

## 4. Open-Source and Democratization of AI

- **OpenAI’s Open-Weight Models**  
  The release of gpt‑oss‑20b and gpt‑oss‑120b marks a significant shift toward openness, enabling local deployment and broader access to powerful AI capabilities. ([windowscentral.com](https://www.windowscentral.com/artificial-intelligence/openai-chatgpt/openai-launches-two-gpt-models-theyre-not-gpt-5-but-they-run-locally-on-snapdragon-pcs-and-nvidia-rtx-gpus?utm_source=openai))

- **Hugging Face SmolLM3 and Liquid AI LFM2**  
  - **SmolLM3**: A 3B-parameter open-source LLM with a 128K token context window, outperforming similar-sized models. ([linkedin.com](https://www.linkedin.com/pulse/top-generative-ai-updates-week-july-2-2025-kalyan-ks-oorrc?utm_source=openai))  
  - **LFM2**: A family of small foundation models (350M–1.2B parameters) optimized for on-device deployment, offering faster training and inference. ([linkedin.com](https://www.linkedin.com/pulse/top-generative-ai-updates-week-july-2-2025-kalyan-ks-oorrc?utm_source=openai))

---

## 5. Real-World Applications and Research

- **Protein Design with Diffusion Models**  
  A 2025 study highlights the success of diffusion-based generative models like RFDiffusion in de novo protein design, achieving higher success rates and lower experimental costs compared to traditional methods. ([arxiv.org](https://arxiv.org/abs/2504.16479?utm_source=openai))

- **Semantic Frameworks for Multimedia AI**  
  A recent academic paper proposes a semantic information-theoretic framework for generative AI in multimedia communication, introducing concepts like semantic entropy and mutual information to improve semantic fidelity. ([arxiv.org](https://arxiv.org/abs/2508.17163?utm_source=openai))

---

## Summary Table

| Area                     | Key Developments                                                                 |
|--------------------------|----------------------------------------------------------------------------------|
| **Models**               | GPT‑5, o4‑mini, Claude 3.7 Sonnet, open-weight models (gpt‑oss)                  |
| **Multimodal AI**        | Nano Banana, Veo 3, Sora, Midjourney V7, Runway Gen‑4, Stable Audio 2.0          |
| **Hardware**             | NVIDIA Blackwell/Rubin chips, Qualcomm AI200/AI250 accelerators                  |
| **Open-Source Access**   | gpt‑oss models, SmolLM3, LFM2                                                    |
| **Applications & Research** | Protein design, semantic multimedia frameworks                                 |

---

### Final Thoughts

Generative AI in 2025 is characterized by:

- **Multimodal integration**: Models now seamlessly handle text, image, audio, and video.
- **Reasoning and autonomy**: Features like “thinking modes” and agentic capabilities are becoming standard.
- **Hardware acceleration**: New chips and accelerators are being developed to meet the demands of generative workloads.
- **Democratization**: Open-weight models and efficient architectures are making powerful AI more accessible.
- **Real-world impact**: From protein design to semantic communication, generative AI is driving innovation across domains.

Let me know if you'd like a deeper dive into any specific model, application, or technology!

# 🧩 Try It Yourself: Two-Step RAG (Private Data + Combined Search)

## Step 1 — Upload & Create Vector Store
1. Upload a short text file (e.g., `my_notes.txt`) to your notebook instance.  
2. Create a **vector store** and **ingest** your uploaded file.  
3. Run a simple test query to verify retrieval:  

In [35]:
with open('pollinator.txt', 'rb') as f:
    file = client.files.create(
        file=f,
        purpose="assistants"
    )

file_id = file.id
print(file_id)

vector_store = client.vector_stores.create(
    name="my_vector_store"
)
vector_store_id = vector_store.id
print(vector_store_id)

attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-HpqBFSgvEVyZpbCHSmQt7m
vs_69126bfcf6f08191b347cb2896693a4a
file-HpqBFSgvEVyZpbCHSmQt7m


In [36]:
query = "the latest development in pollinators"
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

Based on contributions to the literature, the transition will continue 
to  advocate  for  the  role
 Relevant score: 0.7723596846774967
5. Conclusion

A  widespread  decline  in  pollinators  will  render  our  planet  less 
vibrant and
 Relevant score: 0.7571225995489589
Three  separate  studies  examined  the  effects  of  neonicotinoid  in-
secticides  and  proposed  
 Relevant score: 0.7205623461711794
The  quality  of  pollinator  habitats  and  the  complementarity  and 
connectivity among individua
 Relevant score: 0.6836792815552349
The  Federal  Emergency  Management  Agency  (FEMA)  published 
guidelines  on  nature-based  soluti
 Relevant score: 0.6797487724144816


## Step 2 — Combine File Search with Web Search
1. Enable both **file_search** and **web_search** in the Responses API.  
2. Use a prompt that asks the model to merge insights from both sources.  
   > Example: “Using my uploaded notes and the latest web information, summarize the current trends on this topic.”  
3. Review how the answer from your file and **current info** from the web.

✅ You’ve created a RAG system that combines **private** and **public** data for comprehensive, up-to-date analysis.


In [38]:
combined_search_response = client.responses.create(
    model="gpt-4o",
    input= query,
    temperature = 0,
    instructions="Using my uploaded notes and the latest web information, summarize the current trends on this topic",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)
display(Markdown(combined_search_response.output_text))

The latest developments in pollinators focus on addressing the decline through integrated emergency management and alternative pollination strategies. Here are some key trends:

1. **Integrated Emergency Management**: This approach emphasizes collaboration among international organizations, developed, and developing nations to address pollinator decline. It involves coordinated efforts to manage ecological, social, and policy aspects, aiming to create a comprehensive strategy for pollinator health.

2. **Pollinator-Friendly Practices**: These include planting native flora, creating habitats, reducing outdoor lighting, and minimizing lawnmower use. Such practices aim to enhance pollinator habitats and support their populations.

3. **Alternative Pollinators**: With the decline of honeybee populations, there is a push to explore and utilize alternative pollinators. This includes increasing the number of managed pollinators and ensuring they do not pose additional risks.

4. **Global Stewardship and Collaboration**: Efforts are being made to align pollinator protection with global sustainability goals, such as the UN's Sustainable Development Goals. This involves fostering partnerships and information sharing among various stakeholders.

5. **Emergency Training and Exercises**: Initiatives like Australia's "Exercise Bee Prepared" are being implemented to train stakeholders in managing pollinator emergencies and identifying invasive species.

These strategies reflect a growing recognition of the critical role pollinators play in ecosystems and food security, and the need for comprehensive, coordinated efforts to mitigate their decline.